In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle

In [2]:
# path to playlist file
root_path = !pwd
root_path = str(root_path[0])

In [3]:
# filename 
filepath = f"{root_path}/playlist_data/sampledata/"
filename = 'mpd.slice.0-999.json'

In [4]:
# path + filename
fpath_name = f"{filepath}{filename}"
# print(fpath_name)

In [5]:
# Open file of playlist and obtain fields of json file 
with open(fpath_name) as data_file:    
    data = json.load(data_file)  

In [6]:
# # Explore fields in json file
# print("File keys: ", list(data.keys()))
# print("info: ",data[list(data.keys())[0]])
# #print(f"{list(data.keys())[1]} : ",data[list(data.keys())[1]])
# print("playlists: ",list(data['playlists'][0].keys()))
# print("tracks: ", list(data['playlists'][0]['tracks'][0].keys()))

In [7]:
# Extract track features
num_keys_old = 0
for index, playlist in enumerate(data['playlists']):
    num_keys =  len(playlist.keys())
    if index == 0:
        num_key_old = num_keys
        continue
    if num_keys > num_key_old:
        keys = playlist.keys()
    num_key_old = num_keys

In [8]:
# Construct keys of playlist
keys = list(keys)
keys.remove('tracks')
print(" Playlist keys: ", keys)

 Playlist keys:  ['name', 'collaborative', 'pid', 'modified_at', 'num_tracks', 'num_albums', 'num_followers', 'num_edits', 'duration_ms', 'num_artists', 'description']


In [9]:
# Create dataframe with track and playlist info
music_df = pd.json_normalize(data['playlists'],  meta = keys, meta_prefix = 'playlist_', errors='ignore', record_path=['tracks'], record_prefix = 'track_')
music_df.head()

,track_pos,track_artist_name,track_track_uri,track_artist_uri,track_track_name,track_album_uri,track_duration_ms,track_album_name,playlist_name,playlist_collaborative,playlist_pid,playlist_modified_at,playlist_num_tracks,playlist_num_albums,playlist_num_followers,playlist_num_edits,playlist_duration_ms,playlist_num_artists,playlist_description
0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
1,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
2,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
3,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN
4,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,Throwbacks,false,0,1493424000,52,47,1,6,11532414,37,NaN


In [10]:
# Check column values 
music_df[music_df['playlist_pid'] == 0].columns

Index(['track_pos', 'track_artist_name', 'track_track_uri', 'track_artist_uri',
       'track_track_name', 'track_album_uri', 'track_duration_ms',
       'track_album_name', 'playlist_name', 'playlist_collaborative',
       'playlist_pid', 'playlist_modified_at', 'playlist_num_tracks',
       'playlist_num_albums', 'playlist_num_followers', 'playlist_num_edits',
       'playlist_duration_ms', 'playlist_num_artists', 'playlist_description'],
      dtype='object')

In [11]:
# Select indexes of playlists that have a description
playlists_id_with_description = music_df[~music_df['playlist_description'].isna()]['playlist_pid'].unique()
print(playlists_id_with_description)

[94 102 320 329 339 353 354 370 475 491 522 526 548 626 741 767 778 818
 871 925]


In [12]:
# Working only with playlists that contain a "description"
subset_df = music_df[music_df['playlist_pid'].isin(playlists_id_with_description)]
print(subset_df['playlist_pid'].unique())

[94 102 320 329 339 353 354 370 475 491 522 526 548 626 741 767 778 818
 871 925]


In [13]:
num_records = len(subset_df)
num_unique_records = len(subset_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique())
print('Number of table records: ', num_records)
print('Number of unique artist, song pairs: ', num_unique_records )
print('Number of repeated songs: ', num_records - num_unique_records )
print('Number of playlists: ', len(subset_df['playlist_pid'].unique()))
print('Number of unique artists: ', len(subset_df['track_artist_name'].unique()))

Number of table records:  1543
Number of unique artist, song pairs:  1486
Number of repeated songs:  57
Number of playlists:  20
Number of unique artists:  754


In [14]:
# Function to select track_uri ONLY 100 records for now
def get_features(df):
    indexes = []
    audio_features = []
    track_popularity = []
    artist_genre = []
    artist_popularity = []
    album_popularity = []
    start = 0
    for row_index, row in df.iloc[0:100].iterrows():
        #print(row_index, row['track_track_uri'])
        track_uri = row['track_track_uri']
        artist_uri = row['track_artist_uri']
        album_uri = row['track_album_uri']
        if start == 0:
            keys = spotify.audio_features(tracks=track_uri)[0].keys()
        start += 1
    
        track_popularity.append(spotify.track(track_uri)['popularity'])
        
        indexes.append(row_index) 
        audio_features.append(spotify.audio_features(tracks=track_uri)[0].values())
        
        artist_results = spotify.artist(artist_uri)
        
        artist_genre.append(artist_results['genres'])
        artist_popularity.append(artist_results['popularity'])
        #print(artist_results['genres'], artist_results['popularity'])
        album_results = spotify.album(album_uri)
        album_popularity.append(album_results['popularity'])
        
        
    features_df = pd.DataFrame(audio_features, columns=keys)
    features_df['song_popularity'] = track_popularity
    features_df['artist_genre'] = artist_genre
    features_df['artist_popularity']  = artist_popularity
    features_df['album_popularity'] = album_popularity
    features_df['index'] = indexes
    features_df.set_index('index', inplace=True)
    return features_df

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

NameError: name 'SpotifyClientCredentials' is not defined

In [170]:
audio_sp_df = get_features(subset_df)
audio_sp_df.index.name = None
audio_sp_df.drop(['type', 'id', 'track_href', 'analysis_url'], inplace=True, axis=1)
audio_sp_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,uri,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
5431,0.546,0.311,1,-10.396,1,0.0288,0.8180,0.000017,0.1070,0.246,147.464,spotify:track:13HVjjWUZFaWilh2QUJKsP,260533,4,71,"[neo mellow, pop, uk pop]",71,68
5432,0.448,0.431,4,-11.475,0,0.0456,0.8840,0.000009,0.1000,0.240,173.728,spotify:track:4jpwHDjemKbmNy0pyWDHpr,233973,3,0,"[folk-pop, neo mellow]",70,0
5433,0.627,0.699,10,-4.504,1,0.0244,0.1740,0.000000,0.0789,0.535,94.003,spotify:track:1a1i2IwXnUN0PjllVEhA4B,261427,4,34,"[celtic rock, pop]",73,33
5434,0.742,0.479,10,-5.830,1,0.0551,0.2900,0.000000,0.3910,0.346,119.951,spotify:track:7129iqBafaphfc3WPCGC0L,237300,4,53,"[danish pop, scandipop]",70,53
5435,0.343,0.241,0,-10.980,1,0.0367,0.8080,0.000000,0.1070,0.169,184.232,spotify:track:6hsP7b83RjEY51ENfo1ZOc,233240,3,36,[idol],27,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5526,0.515,0.867,11,-5.438,1,0.1060,0.0456,0.000000,0.1380,0.523,129.961,spotify:track:3Vmbt5Yw39WqaIg4eRoOmk,236547,4,47,"[neon pop punk, pop punk, vegas indie]",56,59
5527,0.510,0.760,0,-6.995,1,0.0411,0.1480,0.011600,0.0706,0.550,118.829,spotify:track:5YtxOL4iUchhynLL9nEBwQ,170787,4,0,"[classic rock, glam rock, rock]",85,0
5528,0.692,0.651,9,-8.267,1,0.0324,0.2920,0.002410,0.1050,0.706,97.923,spotify:track:0bRXwKfigvpKZUurwqAlEh,254560,4,80,"[funk, quiet storm, soul]",72,72
5529,0.502,0.156,0,-9.155,1,0.0556,0.8830,0.000000,0.0867,0.352,164.191,spotify:track:0golpXYIyNhpxz9GH1MMTR,224880,4,14,[post-teen pop],54,17


In [198]:
# Parameters to use for the model
audio_sp_df.keys()

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'uri', 'duration_ms', 'time_signature', 'song_popularity',
       'artist_genre', 'artist_popularity', 'album_popularity'],
      dtype='object')

In [244]:
check_df = subset_df.iloc[0:100].merge(audio_sp_df, how='left' , left_on = 'track_track_uri', right_on='uri')
check_df.drop(['track_pos', 'uri', 'mode', 'playlist_duration_ms','playlist_num_albums','playlist_num_artists',  'track_artist_uri', 'track_album_uri', 'track_duration_ms','playlist_num_followers', 'playlist_num_edits', 'playlist_collaborative', 'playlist_modified_at', 'playlist_num_tracks'], inplace = True, axis=1)
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,James Bay,spotify:track:13HVjjWUZFaWilh2QUJKsP,Let It Go,Chaos And The Calm,relax,94,chilllll out,0.546,0.311,1,...,0.000017,0.1070,0.246,147.464,260533,4,71,"[neo mellow, pop, uk pop]",71,68
1,Passenger,spotify:track:4jpwHDjemKbmNy0pyWDHpr,All the Little Lights,All The Little Lights,relax,94,chilllll out,0.448,0.431,4,...,0.000009,0.1000,0.240,173.728,233973,3,0,"[folk-pop, neo mellow]",70,0
2,The Script,spotify:track:1a1i2IwXnUN0PjllVEhA4B,Breakeven,The Script,relax,94,chilllll out,0.627,0.699,10,...,0.000000,0.0789,0.535,94.003,261427,4,34,"[celtic rock, pop]",73,33
3,Lukas Graham,spotify:track:7129iqBafaphfc3WPCGC0L,7 Years,Lukas Graham (Blue Album),relax,94,chilllll out,0.742,0.479,10,...,0.000000,0.3910,0.346,119.951,237300,4,53,"[danish pop, scandipop]",70,53
4,Jason Castro,spotify:track:6hsP7b83RjEY51ENfo1ZOc,Hallelujah,Jason Castro,relax,94,chilllll out,0.343,0.241,0,...,0.000000,0.1070,0.169,184.232,233240,3,36,[idol],27,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Cab,spotify:track:3Vmbt5Yw39WqaIg4eRoOmk,Endlessly,Symphony Soldier,relax,94,chilllll out,0.515,0.867,11,...,0.000000,0.1380,0.523,129.961,236547,4,47,"[neon pop punk, pop punk, vegas indie]",56,59
96,Queen,spotify:track:5YtxOL4iUchhynLL9nEBwQ,You're My Best Friend - Remastered 2011,A Night At The Opera,relax,94,chilllll out,0.510,0.760,0,...,0.011600,0.0706,0.550,118.829,170787,4,0,"[classic rock, glam rock, rock]",85,0
97,Bill Withers,spotify:track:0bRXwKfigvpKZUurwqAlEh,Lovely Day,Menagerie,relax,94,chilllll out,0.692,0.651,9,...,0.002410,0.1050,0.706,97.923,254560,4,80,"[funk, quiet storm, soul]",72,72
98,Grace VanderWaal,spotify:track:0golpXYIyNhpxz9GH1MMTR,Beautiful Thing,Perfectly Imperfect (Japan Version),relax,94,chilllll out,0.502,0.156,0,...,0.000000,0.0867,0.352,164.191,224880,4,14,[post-teen pop],54,17


In [245]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000
mean,0.544840,0.498211,5.090000,-8.251960,0.051032,0.411161,0.011082,0.138284,0.379152,124.198230,225042.670000,3.920000,47.02000,65.220000,48.130000
std,0.117165,0.213460,3.530802,3.872903,0.044053,0.318725,0.084354,0.087923,0.177338,35.150118,41958.409532,0.485757,29.04263,17.799433,28.682045
min,0.209000,0.026200,0.000000,-27.691000,0.024300,0.001860,0.000000,0.036800,0.079700,63.050000,116532.000000,1.000000,0.00000,5.000000,0.000000
25%,0.464750,0.323000,2.000000,-10.126000,0.029150,0.118000,0.000000,0.095250,0.249250,93.760500,195005.500000,4.000000,25.50000,56.000000,29.250000
50%,0.543000,0.504000,5.000000,-7.499000,0.035050,0.296000,0.000000,0.112000,0.355000,123.407500,227619.500000,4.000000,55.50000,68.500000,58.500000
75%,0.628250,0.665750,7.250000,-5.621250,0.050325,0.745250,0.000018,0.151750,0.491750,147.870000,245570.000000,4.000000,71.00000,76.000000,69.250000
max,0.824000,0.928000,11.000000,-3.325000,0.328000,0.964000,0.829000,0.686000,0.955000,205.903000,376827.000000,5.000000,91.00000,100.000000,88.000000


In [246]:
# Make album popularity equal to artist popularity if album popularity is 0
check_df['album_popularity'] = np.where(check_df['album_popularity'] == 0, check_df['artist_popularity'], check_df['album_popularity'])
check_df['song_popularity'] = np.where(check_df['song_popularity'] == 0, check_df['artist_popularity'], check_df['song_popularity'])
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,James Bay,spotify:track:13HVjjWUZFaWilh2QUJKsP,Let It Go,Chaos And The Calm,relax,94,chilllll out,0.546,0.311,1,...,0.000017,0.1070,0.246,147.464,260533,4,71,"[neo mellow, pop, uk pop]",71,68
1,Passenger,spotify:track:4jpwHDjemKbmNy0pyWDHpr,All the Little Lights,All The Little Lights,relax,94,chilllll out,0.448,0.431,4,...,0.000009,0.1000,0.240,173.728,233973,3,70,"[folk-pop, neo mellow]",70,70
2,The Script,spotify:track:1a1i2IwXnUN0PjllVEhA4B,Breakeven,The Script,relax,94,chilllll out,0.627,0.699,10,...,0.000000,0.0789,0.535,94.003,261427,4,34,"[celtic rock, pop]",73,33
3,Lukas Graham,spotify:track:7129iqBafaphfc3WPCGC0L,7 Years,Lukas Graham (Blue Album),relax,94,chilllll out,0.742,0.479,10,...,0.000000,0.3910,0.346,119.951,237300,4,53,"[danish pop, scandipop]",70,53
4,Jason Castro,spotify:track:6hsP7b83RjEY51ENfo1ZOc,Hallelujah,Jason Castro,relax,94,chilllll out,0.343,0.241,0,...,0.000000,0.1070,0.169,184.232,233240,3,36,[idol],27,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Cab,spotify:track:3Vmbt5Yw39WqaIg4eRoOmk,Endlessly,Symphony Soldier,relax,94,chilllll out,0.515,0.867,11,...,0.000000,0.1380,0.523,129.961,236547,4,47,"[neon pop punk, pop punk, vegas indie]",56,59
96,Queen,spotify:track:5YtxOL4iUchhynLL9nEBwQ,You're My Best Friend - Remastered 2011,A Night At The Opera,relax,94,chilllll out,0.510,0.760,0,...,0.011600,0.0706,0.550,118.829,170787,4,85,"[classic rock, glam rock, rock]",85,85
97,Bill Withers,spotify:track:0bRXwKfigvpKZUurwqAlEh,Lovely Day,Menagerie,relax,94,chilllll out,0.692,0.651,9,...,0.002410,0.1050,0.706,97.923,254560,4,80,"[funk, quiet storm, soul]",72,72
98,Grace VanderWaal,spotify:track:0golpXYIyNhpxz9GH1MMTR,Beautiful Thing,Perfectly Imperfect (Japan Version),relax,94,chilllll out,0.502,0.156,0,...,0.000000,0.0867,0.352,164.191,224880,4,14,[post-teen pop],54,17


In [247]:
check_df.describe()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_popularity,album_popularity
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.544840,0.498211,5.090000,-8.251960,0.051032,0.411161,0.011082,0.138284,0.379152,124.198230,225042.670000,3.920000,58.230000,65.220000,59.340000
std,0.117165,0.213460,3.530802,3.872903,0.044053,0.318725,0.084354,0.087923,0.177338,35.150118,41958.409532,0.485757,20.829202,17.799433,19.695264
min,0.209000,0.026200,0.000000,-27.691000,0.024300,0.001860,0.000000,0.036800,0.079700,63.050000,116532.000000,1.000000,3.000000,5.000000,1.000000
25%,0.464750,0.323000,2.000000,-10.126000,0.029150,0.118000,0.000000,0.095250,0.249250,93.760500,195005.500000,4.000000,44.750000,56.000000,51.250000
50%,0.543000,0.504000,5.000000,-7.499000,0.035050,0.296000,0.000000,0.112000,0.355000,123.407500,227619.500000,4.000000,64.500000,68.500000,63.000000
75%,0.628250,0.665750,7.250000,-5.621250,0.050325,0.745250,0.000018,0.151750,0.491750,147.870000,245570.000000,4.000000,73.250000,76.000000,73.000000
max,0.824000,0.928000,11.000000,-3.325000,0.328000,0.964000,0.829000,0.686000,0.955000,205.903000,376827.000000,5.000000,91.000000,100.000000,90.000000


In [248]:
check_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 99
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   track_artist_name     100 non-null    object 
 1   track_track_uri       100 non-null    object 
 2   track_track_name      100 non-null    object 
 3   track_album_name      100 non-null    object 
 4   playlist_name         100 non-null    object 
 5   playlist_pid          100 non-null    object 
 6   playlist_description  100 non-null    object 
 7   danceability          100 non-null    float64
 8   energy                100 non-null    float64
 9   key                   100 non-null    int64  
 10  loudness              100 non-null    float64
 11  speechiness           100 non-null    float64
 12  acousticness          100 non-null    float64
 13  instrumentalness      100 non-null    float64
 14  liveness              100 non-null    float64
 15  valence               10

In [250]:
# NLP workflow
# Need to vectorize string fields
# Use all string fields and combine them in a column containing all the words 

In [251]:
check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ' '.join([i.replace('-', ' ').replace('_', ' ') for i in x]))
# check_df['artist_genre'] = check_df['artist_genre'].apply(lambda x: ''.join([i.replace('', '_') for i in x]))

In [252]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity
0,James Bay,spotify:track:13HVjjWUZFaWilh2QUJKsP,Let It Go,Chaos And The Calm,relax,94,chilllll out,0.546,0.311,1,...,0.000017,0.1070,0.246,147.464,260533,4,71,neo mellow pop uk pop,71,68
1,Passenger,spotify:track:4jpwHDjemKbmNy0pyWDHpr,All the Little Lights,All The Little Lights,relax,94,chilllll out,0.448,0.431,4,...,0.000009,0.1000,0.240,173.728,233973,3,70,folk pop neo mellow,70,70
2,The Script,spotify:track:1a1i2IwXnUN0PjllVEhA4B,Breakeven,The Script,relax,94,chilllll out,0.627,0.699,10,...,0.000000,0.0789,0.535,94.003,261427,4,34,celtic rock pop,73,33
3,Lukas Graham,spotify:track:7129iqBafaphfc3WPCGC0L,7 Years,Lukas Graham (Blue Album),relax,94,chilllll out,0.742,0.479,10,...,0.000000,0.3910,0.346,119.951,237300,4,53,danish pop scandipop,70,53
4,Jason Castro,spotify:track:6hsP7b83RjEY51ENfo1ZOc,Hallelujah,Jason Castro,relax,94,chilllll out,0.343,0.241,0,...,0.000000,0.1070,0.169,184.232,233240,3,36,idol,27,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Cab,spotify:track:3Vmbt5Yw39WqaIg4eRoOmk,Endlessly,Symphony Soldier,relax,94,chilllll out,0.515,0.867,11,...,0.000000,0.1380,0.523,129.961,236547,4,47,neon pop punk pop punk vegas indie,56,59
96,Queen,spotify:track:5YtxOL4iUchhynLL9nEBwQ,You're My Best Friend - Remastered 2011,A Night At The Opera,relax,94,chilllll out,0.510,0.760,0,...,0.011600,0.0706,0.550,118.829,170787,4,85,classic rock glam rock rock,85,85
97,Bill Withers,spotify:track:0bRXwKfigvpKZUurwqAlEh,Lovely Day,Menagerie,relax,94,chilllll out,0.692,0.651,9,...,0.002410,0.1050,0.706,97.923,254560,4,80,funk quiet storm soul,72,72
98,Grace VanderWaal,spotify:track:0golpXYIyNhpxz9GH1MMTR,Beautiful Thing,Perfectly Imperfect (Japan Version),relax,94,chilllll out,0.502,0.156,0,...,0.000000,0.0867,0.352,164.191,224880,4,14,post teen pop,54,17


In [419]:
label_columns = ['track_track_name', 'artist_genre']
check_df[label_columns]

,track_track_name,artist_genre
0,Let It Go,neo mellow pop uk pop
1,All the Little Lights,folk pop neo mellow
2,Breakeven,celtic rock pop
3,7 Years,danish pop scandipop
4,Hallelujah,idol
...,...,...
95,Endlessly,neon pop punk pop punk vegas indie
96,You're My Best Friend - Remastered 2011,classic rock glam rock rock
97,Lovely Day,funk quiet storm soul
98,Beautiful Thing,post teen pop


In [550]:
# Combine 5 fields into one
string_field = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
string_field = string_field.replace({"[^A-Za-z ]+": ""}, regex = True)
#print('last song: ',string_field.tail())
test_song = string_field[0:int(len(string_field)*0.4)]
print(test_song)
string_field = string_field[int(len(string_field)*0.4):]
string_field

0                       Let It Go neo mellow pop uk pop
1             All the Little Lights folk pop neo mellow
2                             Breakeven celtic rock pop
3                            Years danish pop scandipop
4                                       Hallelujah idol
5     A Drop In The Ocean neo mellow piano rock vira...
6                      The Scientist permanent wave pop
7                      Big Yellow Taxi canadian country
8     How to Save a Life neo mellow piano rock pop p...
9                               fOoL fOr YoU pop uk pop
10                           Sunrise american metalcore
11                                I See Fire pop uk pop
12                                Lego House pop uk pop
13               boy band pop post teen pop talent show
14    Me Myself  I indie pop rap oakland hip hop pop...
15          I Was Made For Loving You pop post teen pop
16                  Unbreakable Smile pop post teen pop
17    Sweet Caroline adult standards brill build

40    Im Like A Bird  Acoustic Version canadian lati...
41    Forever Young new romantic new wave new wave p...
42    True Colors Glee Cast Version glee club post t...
43    Smile Glee Cast Version  Cover of Lily Allen S...
44                           Fix You permanent wave pop
45    Chasing Cars irish rock modern rock neo mellow...
46                              When The Stars Go Blue 
47                      Thats Whats Up canadian country
48                                 Girl Crush viral pop
49    Telephone Line album rock art rock beatlesque ...
50                   More Than A Band pop post teen pop
51    Fall For You neon pop punk piano rock pixie po...
52               Drops of Jupiter channel pop viral pop
53    All You Ever contemporary country country coun...
54                   YOUTH australian pop pop viral pop
55                                                     
56            The Man Who Cant Be Moved celtic rock pop
57                  Ride modern rock pop pov ind

In [580]:
check_df['All_text'] = check_df.track_track_name.str.cat(" " + check_df.artist_genre)
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity,All_text,ClusterPrediction
0,James Bay,spotify:track:13HVjjWUZFaWilh2QUJKsP,Let It Go,Chaos And The Calm,relax,94,chilllll out,0.546,0.311,1,...,0.246,147.464,260533,4,71,neo mellow pop uk pop,71,68,Let It Go neo mellow pop uk pop,
1,Passenger,spotify:track:4jpwHDjemKbmNy0pyWDHpr,All the Little Lights,All The Little Lights,relax,94,chilllll out,0.448,0.431,4,...,0.240,173.728,233973,3,70,folk pop neo mellow,70,70,All the Little Lights folk pop neo mellow,
2,The Script,spotify:track:1a1i2IwXnUN0PjllVEhA4B,Breakeven,The Script,relax,94,chilllll out,0.627,0.699,10,...,0.535,94.003,261427,4,34,celtic rock pop,73,33,Breakeven celtic rock pop,
3,Lukas Graham,spotify:track:7129iqBafaphfc3WPCGC0L,7 Years,Lukas Graham (Blue Album),relax,94,chilllll out,0.742,0.479,10,...,0.346,119.951,237300,4,53,danish pop scandipop,70,53,7 Years danish pop scandipop,
4,Jason Castro,spotify:track:6hsP7b83RjEY51ENfo1ZOc,Hallelujah,Jason Castro,relax,94,chilllll out,0.343,0.241,0,...,0.169,184.232,233240,3,36,idol,27,33,Hallelujah idol,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Cab,spotify:track:3Vmbt5Yw39WqaIg4eRoOmk,Endlessly,Symphony Soldier,relax,94,chilllll out,0.515,0.867,11,...,0.523,129.961,236547,4,47,neon pop punk pop punk vegas indie,56,59,Endlessly neon pop punk pop punk vegas indie,
96,Queen,spotify:track:5YtxOL4iUchhynLL9nEBwQ,You're My Best Friend - Remastered 2011,A Night At The Opera,relax,94,chilllll out,0.510,0.760,0,...,0.550,118.829,170787,4,85,classic rock glam rock rock,85,85,You're My Best Friend - Remastered 2011 classi...,
97,Bill Withers,spotify:track:0bRXwKfigvpKZUurwqAlEh,Lovely Day,Menagerie,relax,94,chilllll out,0.692,0.651,9,...,0.706,97.923,254560,4,80,funk quiet storm soul,72,72,Lovely Day funk quiet storm soul,
98,Grace VanderWaal,spotify:track:0golpXYIyNhpxz9GH1MMTR,Beautiful Thing,Perfectly Imperfect (Japan Version),relax,94,chilllll out,0.502,0.156,0,...,0.352,164.191,224880,4,14,post teen pop,54,17,Beautiful Thing post teen pop,


In [581]:
# Imports
from sklearn.cluster import KMeans
from sklearn import metrics 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

In [582]:
# import nltk
# from nltk.stem import WordNetLemmatizer

In [583]:
# stopwords = nltk.corpus.stopwords.words('english')
# lemmatizer = WordNetLemmatizer()
# nltk.download('stopwords')

In [584]:
vectorizer = TfidfVectorizer(stop_words='english')

X_train = vectorizer.fit_transform(string_field)
print(X_train)
print(f"n_samples: {X_train.shape[0]}, n_features: {X_train.shape[1]}")

  (0, 43)	0.2139071320194041
  (0, 109)	0.3024435884327589
  (0, 81)	0.32432978437262566
  (0, 23)	0.5891246900615462
  (0, 147)	0.24367457136125664
  (0, 0)	0.23235751615847175
  (0, 17)	0.32432978437262566
  (0, 84)	0.32432978437262566
  (0, 74)	0.2945623450307731
  (1, 137)	0.2764458767943101
  (1, 150)	0.3646519531226344
  (1, 120)	0.25107328918352995
  (1, 102)	0.7532198675505899
  (1, 160)	0.2764458767943101
  (1, 58)	0.2764458767943101
  (1, 109)	0.08593031641835683
  (2, 139)	0.21869118218563463
  (2, 111)	0.22689597079618146
  (2, 35)	0.3124487183049803
  (2, 28)	0.3124487183049803
  (2, 64)	0.6248974366099606
  (2, 37)	0.34402369191069154
  (2, 145)	0.34402369191069154
  (2, 109)	0.10693617515334736
  (2, 147)	0.25847094440189267
  :	:
  (55, 101)	0.33187139938435695
  (55, 109)	0.24471337736269533
  (56, 116)	0.3686456494002015
  (56, 59)	0.3686456494002015
  (56, 15)	0.3686456494002015
  (56, 161)	0.3686456494002015
  (56, 63)	0.33481083824221475
  (56, 34)	0.27696984652689

In [585]:
print(f"{X_train.nnz / np.prod(X_train.shape):.3f}")
print(f"About {X_train.nnz / np.prod(X_train.shape)*100:.3f}\% of the entries in the matrix are non zero")

0.037
About 3.681\% of the entries in the matrix are non zero


In [586]:
true_k = 5

model = KMeans(n_clusters=true_k, init='k-means++', max_iter=500, n_init=100)
model.fit(X_train)
print(model.labels_)
print(len(model.labels_))

[4 2 3 3 2 1 0 0 0 1 3 1 0 0 0 0 1 1 4 3 0 4 2 1 2 3 0 0 2 1 0 0 0 0 1 0 0
 1 0 0 0 0 0 2 4 0 4 1 4 4 2 4 1 1 4 0 1 0 3 3]
60


In [587]:
print('Top terms per cluster')
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names_out()
#print(terms)
for i in range(true_k):
    print(f"Cluster {i}: ", end="")
    for ind in order_centroids[i,:10]:
        print(f"{terms[ind]} ", end="")
    print()

Top terms per cluster
Cluster 0: pop viral country dance channel indie rap heart chivalry stars 
Cluster 1: rock piano pop mellow modern classic pov neo soft gold 
Cluster 2: wave permanent new pop army everglow fix yellow romantic rock 
Cluster 3: post teen pop glee band thing light cast club ordinary 
Cluster 4: pop neo acoustic mellow dance rock love lilith xo canadian 


In [633]:
#print(list(test_song[:1]))
#print(list(test_song)[0])
X_test = vectorizer.transform(list(test_song))
#print(X_test)
print(f"n_samples: {X_test.shape[0]}, n_features: {X_test.shape[1]}")
# feature_names = vectorizer.get_feature_names_out()
# print(feature_names)
prediction = model.predict(X_test)
print(prediction)

n_samples: 40, n_features: 163
[4 4 1 0 0 1 2 0 1 4 0 4 4 3 0 3 3 1 1 0 0 4 4 0 1 3 0 0 0 0 1 0 0 4 0 0 0
 4 0 0]


In [589]:
print(model.labels_)
print(prediction)

[4 2 3 3 2 1 0 0 0 1 3 1 0 0 0 0 1 1 4 3 0 4 2 1 2 3 0 0 2 1 0 0 0 0 1 0 0
 1 0 0 0 0 0 2 4 0 4 1 4 4 2 4 1 1 4 0 1 0 3 3]
[4 4 1 0 0 1 2 0 1 4 0 4 4 3 0 3 3 1 1 0 0 4 4 0 1 3 0 0 0 0 1 0 0 4 0 0 0
 4 0 0]


In [590]:
# # Save machine learning model
# filename = 'finalized_model.sav'
# pickle.dump(model, open(filename, 'wb'))

In [591]:
# # load model only once
# with open('finalized_model.sav', 'rb') as fid:
#     model = pickle.load(fid)

In [603]:
check_df['ClusterPrediction'] = ""
check_df['All_text']

0                       Let It Go neo mellow pop uk pop
1             All the Little Lights folk pop neo mellow
2                             Breakeven celtic rock pop
3                          7 Years danish pop scandipop
4                                       Hallelujah idol
                            ...                        
95         Endlessly neon pop punk pop punk vegas indie
96    You're My Best Friend - Remastered 2011 classi...
97                     Lovely Day funk quiet storm soul
98                        Beautiful Thing post teen pop
99                          Light The Sky post teen pop
Name: All_text, Length: 100, dtype: object

In [664]:
def cluster_predict(str_input):
    #print(test_song)
    if isinstance(str_input, str):
        # It is a single string element thus we cannot use list() and have to use brackets [] to transform to list
        X_test = vectorizer.transform([str_input])
        prediction = model.predict(X_test)
    else:
        # if is not a single string element so we can transform into a list directly using list()
        X_test = vectorizer.transform(list(str_input))
        prediction = model.predict(X_test)
    return int(prediction)

In [665]:
check_df['All_text'][0]

'Let It Go neo mellow pop uk pop'

In [666]:
cluster_predict(check_df['All_text'][0])

4

In [667]:
cluster_predict([check_df['All_text'][0]])

4

In [668]:
result = check_df['All_text'].apply(lambda x: cluster_predict(x))
print(result)

0     4
1     4
2     1
3     0
4     0
     ..
95    0
96    1
97    0
98    3
99    3
Name: All_text, Length: 100, dtype: int64


In [669]:
# for i in check_df['All_text']:
#     pred = cluster_predict(i)
#     print(i, pred)
check_df['ClusterPrediction'] = check_df['All_text'].apply(lambda x: cluster_predict(x))

In [670]:
check_df

,track_artist_name,track_track_uri,track_track_name,track_album_name,playlist_name,playlist_pid,playlist_description,danceability,energy,key,...,valence,tempo,duration_ms,time_signature,song_popularity,artist_genre,artist_popularity,album_popularity,All_text,ClusterPrediction
0,James Bay,spotify:track:13HVjjWUZFaWilh2QUJKsP,Let It Go,Chaos And The Calm,relax,94,chilllll out,0.546,0.311,1,...,0.246,147.464,260533,4,71,neo mellow pop uk pop,71,68,Let It Go neo mellow pop uk pop,4
1,Passenger,spotify:track:4jpwHDjemKbmNy0pyWDHpr,All the Little Lights,All The Little Lights,relax,94,chilllll out,0.448,0.431,4,...,0.240,173.728,233973,3,70,folk pop neo mellow,70,70,All the Little Lights folk pop neo mellow,4
2,The Script,spotify:track:1a1i2IwXnUN0PjllVEhA4B,Breakeven,The Script,relax,94,chilllll out,0.627,0.699,10,...,0.535,94.003,261427,4,34,celtic rock pop,73,33,Breakeven celtic rock pop,1
3,Lukas Graham,spotify:track:7129iqBafaphfc3WPCGC0L,7 Years,Lukas Graham (Blue Album),relax,94,chilllll out,0.742,0.479,10,...,0.346,119.951,237300,4,53,danish pop scandipop,70,53,7 Years danish pop scandipop,0
4,Jason Castro,spotify:track:6hsP7b83RjEY51ENfo1ZOc,Hallelujah,Jason Castro,relax,94,chilllll out,0.343,0.241,0,...,0.169,184.232,233240,3,36,idol,27,33,Hallelujah idol,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,The Cab,spotify:track:3Vmbt5Yw39WqaIg4eRoOmk,Endlessly,Symphony Soldier,relax,94,chilllll out,0.515,0.867,11,...,0.523,129.961,236547,4,47,neon pop punk pop punk vegas indie,56,59,Endlessly neon pop punk pop punk vegas indie,0
96,Queen,spotify:track:5YtxOL4iUchhynLL9nEBwQ,You're My Best Friend - Remastered 2011,A Night At The Opera,relax,94,chilllll out,0.510,0.760,0,...,0.550,118.829,170787,4,85,classic rock glam rock rock,85,85,You're My Best Friend - Remastered 2011 classi...,1
97,Bill Withers,spotify:track:0bRXwKfigvpKZUurwqAlEh,Lovely Day,Menagerie,relax,94,chilllll out,0.692,0.651,9,...,0.706,97.923,254560,4,80,funk quiet storm soul,72,72,Lovely Day funk quiet storm soul,0
98,Grace VanderWaal,spotify:track:0golpXYIyNhpxz9GH1MMTR,Beautiful Thing,Perfectly Imperfect (Japan Version),relax,94,chilllll out,0.502,0.156,0,...,0.352,164.191,224880,4,14,post teen pop,54,17,Beautiful Thing post teen pop,3


In [663]:
# for seed in range(5):
#     model = KMeans(
#         n_clusters=true_k,
#         max_iter=500,
#         n_init=1,
#         random_state=seed,
#     ).fit(X)
#     cluster_ids, cluster_sizes = np.unique(model.labels_, return_counts=True)
#     print(f"Number of elements asigned to each cluster: {cluster_sizes}")
# print()

In [720]:
def recommend_util(artist_name, song_name):
    
    # Predict category of input string category
    chosen_song_df = check_df.loc[(check_df['track_artist_name'] == artist_name) & (check_df['track_track_name'] == song_name)]
    str_input =chosen_song_df.track_track_name.str.cat(" " + chosen_song_df.artist_genre)
        
    prediction_inp = cluster_predict(str_input)
    prediction_inp = int(prediction_inp)
    
    temp_df = check_df.loc[check_df['ClusterPrediction'] == prediction_inp]
    new_temp_df = temp_df.sample(5)
    
    return chosen_song_df[['track_artist_name', 'track_track_name']], new_temp_df[['track_artist_name', 'track_track_name']]

In [729]:
song_choice = 2
print(check_df['track_artist_name'][song_choice], check_df['track_track_name'][song_choice])
temp_df = check_df.loc[(check_df['track_artist_name'] == check_df['track_artist_name'][0]) & (check_df['track_track_name'] == check_df['track_track_name'][0])]
string_input = temp_df.track_track_name.str.cat(" " + temp_df.artist_genre)
string_input
prediction_inp = cluster_predict(string_input)
print(prediction_inp)

The Script Breakeven
4


In [734]:
song_choice = 1
original_song , recommended_songs = recommend_util(check_df['track_artist_name'][song_choice], check_df['track_track_name'][song_choice])
print(original_song)
print('\n')
print(recommended_songs)

  track_artist_name       track_track_name
1         Passenger  All the Little Lights


   track_artist_name                    track_track_name
86        Jason Mraz                        Love Someone
40     Nelly Furtado  I'm Like A Bird - Acoustic Version
88         Parachute                         She Is Love
58         Howie Day          Collide - Acoustic Version
94             Train              When I Look to the Sky


In [228]:
# Example of how to pull metadata from a single track 
spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

Row = 5 

track_uri = subset_df.iloc[Row]['track_track_uri']
artist_uri = subset_df.iloc[Row]['track_artist_uri']
album_uri = subset_df.iloc[Row]['track_album_uri']

print('ARTIST: ', subset_df.iloc[Row]['track_artist_name'])
print('ALBUM: ', subset_df.iloc[Row]['track_album_name'])
print('SONG: ', subset_df.iloc[Row]['track_track_name'])
track_popularity = spotify.track(track_uri)['popularity']
print('track_popularity: ', track_popularity)
track_results = spotify.audio_features(tracks=track_uri)
print('audio_features: ', track_results[0])
artist_results = spotify.artist(artist_uri)
print('artist_genre: ', artist_results['genres'])
print('artist_popularity: ', artist_results['popularity'])
artist_albums_results = spotify.artist_albums(artist_uri)
#print(artist_albums_results)
album_results = spotify.album(album_uri)
print('album_popularity: ',album_results['popularity'])
print('album_keys: ', album_results.keys())
# print(artist_results['genres'])
# print(artist_results['popularity'])

ARTIST:  Ron Pope
ALBUM:  The Bedroom Demos
SONG:  A Drop In The Ocean
track_popularity:  67
audio_features:  {'danceability': 0.447, 'energy': 0.393, 'key': 5, 'loudness': -8.65, 'mode': 1, 'speechiness': 0.038, 'acousticness': 0.785, 'instrumentalness': 0, 'liveness': 0.28, 'valence': 0.564, 'tempo': 73.139, 'type': 'audio_features', 'id': '5JDcQAztvZTIkrWoZihgvC', 'uri': 'spotify:track:5JDcQAztvZTIkrWoZihgvC', 'track_href': 'https://api.spotify.com/v1/tracks/5JDcQAztvZTIkrWoZihgvC', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/5JDcQAztvZTIkrWoZihgvC', 'duration_ms': 220239, 'time_signature': 3}
artist_genre:  ['neo mellow', 'piano rock', 'viral pop']
artist_popularity:  53
album_popularity:  59
album_keys:  dict_keys(['album_type', 'artists', 'available_markets', 'copyrights', 'external_ids', 'external_urls', 'genres', 'href', 'id', 'images', 'label', 'name', 'popularity', 'release_date', 'release_date_precision', 'total_tracks', 'tracks', 'type', 'uri'])


In [229]:
# Work only with important features
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_duration_ms', 'playlist_description']

In [ ]:
artist_count_df = subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort=False).value_counts(sort=False).reset_index(name="artist_count")

In [ ]:
artist_count_df

In [ ]:
subset_df[['playlist_name', 'track_artist_name']]

In [ ]:
subset_df[['playlist_name','track_artist_name']].groupby(['playlist_name'], sort = False).value_counts(sort=False).reset_index(name="artist_count").info()

In [ ]:
music_df = pd.merge(subset_df, artist_count_df,  how='left', on = ['playlist_name','track_artist_name'])

In [ ]:
music_df

In [ ]:
# subset_df.join(.sum(), on='playlist_name', rsuffix='_count')
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1)

In [ ]:
important_features.append("artist_count")

In [ ]:
music_df[important_features]

In [ ]:
music_df['artist_fraction'] =  music_df['artist_count'] / music_df['playlist_num_tracks'] 
music_df

In [ ]:
subset_df.groupby(['playlist_name','track_artist_name']).size().unstack(fill_value=0)

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).value_counts() 

In [ ]:
vc = music_df[music_df['playlist_num_tracks'] == Ntracks][['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x), axis=1).value_counts()
vc[vc > 1].sum() - len(vc[vc > 1])

In [ ]:
# for i in df[['track_artist_name','track_track_name']].apply(lambda x: ',.'.join(x),axis=1).unique():
# #     print(i.split(',.'))
#     if len(i.split(',.')) == 2:
#         print(i)

In [ ]:
np.unique(music_df[music_df['playlist_num_tracks'] == Ntracks]['track_track_name'])

In [ ]:
important_features = ['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists']

In [ ]:
music_df[music_df['playlist_num_tracks'] == 20][['track_artist_name', 'track_track_name', 'playlist_name', 'playlist_num_tracks', 'playlist_num_albums', 'playlist_duration_ms', 'playlist_num_artists', 'playlist_description']]

In [ ]:
music_df[music_df['playlist_num_tracks'] == Ntracks][important_features].groupby('playlist_name')['track_artist_name'].value_counts()